In [1]:
from __future__ import print_function

#%% import public modules
import sys
sys.path.append('..')

import networkx as nx
import itertools
import os
import random
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import autograd.numpy as np
import copy 

import logging
logging.getLogger().setLevel(logging.CRITICAL)

#%% modules for interactivity
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import GridspecLayout
from ipywidgets import FloatSlider, BoundedFloatText, SelectionSlider

#%% import custom ASOPE modules
import config.config as configuration
from lib.functions import scale_units

from problems.example.evaluator import Evaluator
from problems.example.evolver import Evolver
from problems.example.graph import Graph
from problems.example.assets.propagator import Propagator
from problems.example.assets.functions import psd_, power_, fft_, ifft_

from problems.example.evaluator_subclasses.evaluator_rfawg import RadioFrequencyWaveformGeneration
from problems.example.evaluator_subclasses.evaluator_pulserep import PulseRepetition

from problems.example.node_types_subclasses.inputs import PulsedLaser, ContinuousWaveLaser
from problems.example.node_types_subclasses.outputs import MeasurementDevice
from problems.example.node_types_subclasses.single_path import CorningFiber, PhaseModulator, WaveShaper, DelayLine, ProgrammableFilter
from problems.example.node_types_subclasses.single_path import IntensityModulator, VariableOpticalAttenuator, EDFA
from problems.example.node_types_subclasses.multi_path import VariablePowerSplitter

C:\Users\benjamin\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\benjamin\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\benjamin\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


The following cell is the main interactive plotting functions - they should not need changing. All parameters (unlocked parameters) are extracted and a continuous slider is made, which upper and lower limits, names, and step sizes according to the component model.

In [2]:
def run_interactive_propagation(**kwargs):
    tmp_parameters = copy.deepcopy(attributes['parameters'])
    for i, (parameter_id, parameter) in enumerate(zip(attributes['parameters_uuid'], kwargs.values())):
        ind = attributes['parameters_uuid'].index(parameter_id)
        tmp_parameters[ind] = parameter
    graph.distribute_parameters_from_list(tmp_parameters, attributes['node_edge_index'], attributes['parameter_index'])

    graph.propagate(propagator, save_transforms=True)
    state = graph.measure_propagator(graph.get_output_node())

    fig, axs = create_plot_window(propagator)
    axs[0,0].plot(propagator.t, power_(state))
    axs[0,1].plot(propagator.f, psd_(state, propagator.dt, propagator.df))

    graph.visualize_transforms_dof(axs[1,0], propagator, dof='t', label_verbose=0)
    graph.visualize_transforms_dof(axs[1,1], propagator, dof='f', label_verbose=0)
    #for ax in axs[:,0]:
    #    scale_units(ax, unit='s')
    #for ax in axs[:,1]:
    #    scale_units(ax, unit='Hz')
    plt.show()
    return

def setup_interactivity():
    attributes_to_get = ['parameters', 'parameter_names', 'parameter_symbols',
                         'lower_bounds', 'upper_bounds', 'step_sizes',
                         'parameter_imprecisions','parameters_uuid']
    attributes = graph.extract_attributes_to_list_experimental(attributes_to_get, get_location_indices=True)

    grid_size = int(np.ceil(np.sqrt(len(attributes['parameters']))))
    grid = GridspecLayout(grid_size, grid_size)
    
    sliders = {}
    for i in range(len(attributes['parameters'])):
        """
        fs = FloatSlider(value=attributes['parameters'][i],
                         min=attributes['lower_bounds'][i],
                         max=attributes['upper_bounds'][i],
                         step=attributes['step_sizes'][i] if attributes['step_sizes'][i] is not None else (attributes['upper_bounds'][i]-attributes['lower_bounds'][i])/100,
                         description=attributes['parameter_names'][i],
                         disabled=False,
                         continuous_update=False,
                         orientation='horizontal',
                         readout=True,
                         readout_format='1.2e'
                         )
        """
        num_divs = 100
        step = (attributes['upper_bounds'][i]-attributes['lower_bounds'][i])/num_divs
        options = [attributes['lower_bounds'][i] + step*k for k in range(num_divs)]
        #"""["{:1.2e}".format(option) for option in options]
        fs = SelectionSlider(options=options,
                             value=options[1],
                             description=attributes['parameter_names'][i],
                             disabled=False,
                             continuous_update=False,
                             orientation='horizontal',
                             readout=True)
        #"""
        sliders[str(attributes['parameters_uuid'][i])] = fs
        
    return sliders, attributes


def create_plot_window(propagator):
    fig, axs = plt.subplots(2,2, figsize=[12, 8], gridspec_kw={'wspace':0.4, 'hspace':0.4})

    tlims, flims = [np.min(propagator.t), np.max(propagator.t)], [np.min(propagator.f), np.max(propagator.f)]

    axs[0,0].set(xlabel='Time', ylabel='AU', xlim=np.array(tlims)/tzoom)
    axs[0,1].set(xlabel='Frequency', ylabel='AU', xlim=np.array(flims)/fzoom)
 
    axs[1,0].set(xlabel='Time', ylabel='AU', xlim=np.array(tlims)/tzoom)
    axs[1,1].set(xlabel='Frequency', ylabel='AU', xlim=np.array(flims)/fzoom)
    
    return fig, axs

In [3]:
## create optical system
propagator = Propagator(window_t = 100e-9, n_samples = 2**16, central_wl=1.55e-6)

nodes = {0: ContinuousWaveLaser(),
         1: CorningFiber(),
         2:EDFA(),
        -1: MeasurementDevice()}
edges = [(0,1),(1,2), (2,-1)]

graph = Graph(nodes, edges, propagate_on_edges = False)
graph.assert_number_of_edges()

tzoom, fzoom = 1, 3

## set up interactivity
sliders, attributes = setup_interactivity()

## run interactivity
manual = False
if manual == True:
    interact_manual(run_interactive_propagation, **sliders)
else:
    interact(run_interactive_propagation, **sliders)

interactive(children=(SelectionSlider(continuous_update=False, description='peak_power', index=1, options=(1e-…